In [1]:
from pickling import load_pickle, create_pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import researchpy as rp
import statsmodels.api as sm
import choplo_statistique as cstats
import choplo_visualization as cvis
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

In [2]:
a_df = load_pickle('amp')
a_side_df = load_pickle('side_amp')
a_jt_df = load_pickle('amp_jt_new_filt')
a_jt_full_df = load_pickle('amp_not_norm_joint_df_new_filt')
a_angle_full_df = load_pickle('amp_not_norm_angle_df_new_filt')
a_st_df = load_pickle('amp_strat_new_filt')
a_missing_df = load_pickle('missing_amp')
a_sum_strat_df = load_pickle('a_sum_strat_df')


v_df = load_pickle('vit')
v_side_df = load_pickle('side_vit')
v_jt_df = load_pickle('vit_jt_new_filt')
v_st_df = load_pickle('vit_strat_new_filt')
v_jt_full_df = load_pickle('vit_not_norm_joint_df_new_filt')
v_angle_full_df = load_pickle('vit_not_norm_angle_df_new_filt')
v_missing_df = load_pickle('missing_vit') 
v_sum_strat_df = load_pickle('v_sum_strat_df')

idx = pd.IndexSlice


C:\Users\tousi\AppData\Local\conda\conda\envs\python37\lib\site-packages\pandas\core\indexes\base.py:162: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return cls.__new__(cls, **d)


In [3]:
col0_names = ['Niveau 1', 'Niveau 2', 'Niveau 3']
col1_names = ['Rang', 'Moy (ÉT)', '[Min ; Max]']
ind0_names = ['Phase 1', 'Phase 2', 'Phase 3', 'Phase 4']
ind1_names = ['DPIap', 'DPIp', 'PS', 'SPI', 'ST']


In [5]:
def _rank(s):
    
    s1 = s['phase_1'].rank(ascending=0).astype(int)
    s2 = s['phase_2'].rank(ascending=0).astype(int)
    s3 = s['phase_3'].rank(ascending=0).astype(int)
    s4 = s['phase_4'].rank(ascending=0).astype(int)


    return pd.concat([s1,s2,s3,s4], keys=['phase_1', 'phase_2', 'phase_3', 'phase_4'])

def add_ranks_col(df):
    df['lvl_0','rank'] = _rank(df['lvl_0','mean'])
    df['lvl_1','rank'] = _rank(df['lvl_1','mean'])
    df['lvl_2','rank'] = _rank(df['lvl_2','mean'])

def format_mean_std(m, s):
    mstd_string = "{mean:.2f} ({std:.2f})"
    output = []
    n = len(m)
    for i in range(0,n):
        output.append(mstd_string.format(mean=m[i], std=s[i]))
    return np.array(output)

def format_range(m, M):
    range_string = "[{_min:.2f} ; {_max:.2f}]"
    output = []
    n = len(m)
    for i in range(0,n):
        output.append(range_string.format(_min=m[i], _max=M[i]))
    return np.array(output)

def format_rank(r):
    rank_string = "# {rank}"
    output = []
    n = len(r)
    for i in range(0,n):
        output.append(rank_string.format(rank=r[i]))
    output = pd.Series(output)
    output
    return np.array(output)

def format_df_output(df):
        
    c0 = ['lvl_0', 'lvl_1', 'lvl_2']
    c1 = ['rank', 'm_std', 'range']
    columns = pd.MultiIndex.from_product([c0,c1])
    index = df.index
    
    data = np.empty([len(df.index), len(c0)*len(c1)])
    data[:] = np.nan
    df_output = pd.DataFrame(data, index=index, columns=columns)
    
    for lvl in c0:
        df_output[lvl,'rank'] = format_rank(df[lvl,'rank'])
        df_output[lvl,'m_std'] = format_mean_std(df[lvl,'mean'], df[lvl,'std'])
        df_output[lvl,'range'] = format_range(df[lvl,'min'], df[lvl,'max'])
    
    return df_output
    

In [48]:
v_sum_strat_df.index.set_names(['level','players', 'moves'],inplace=True)
v = v_sum_strat_df.groupby(['level','players']).mean().groupby('level').agg(['mean','std','min','max']).transpose().unstack(2)
add_ranks_col(v)
v.sort_index(axis=1, inplace=True)

In [49]:
v.index.get_level_values(1).unique()

Index(['DPIap', 'DPIp', 'PS', 'SPI', 'ST'], dtype='object')

In [50]:
len(v.index)

20

In [63]:
v_output = format_df_output(v)
colnames = pd.MultiIndex.from_product([col0_names,col1_names])
index = pd.MultiIndex.from_product([ind0_names, ind1_names])
v_output.columns = colnames
v_output.index = index

In [66]:
v_output.to_excel('table_strat_vitesse.xlsx')

In [7]:
def create_and_save_df_strat(sum_df, output_name):
    col0_names = ['Niveau 1', 'Niveau 2', 'Niveau 3']
    col1_names = ['Rang', 'Moy (ÉT)', '[Min ; Max]']
    ind0_names = ['Phase 1', 'Phase 2', 'Phase 3', 'Phase 4']
    ind1_names = ['DPIap', 'DPIp', 'PS', 'SPI', 'ST']

    sum_df.index.set_names(['level','players', 'moves'],inplace=True)
    df = sum_df.groupby(['level','players']).mean().groupby('level').agg(['mean','std','min','max']).transpose().unstack(2)
    add_ranks_col(df)
    df.sort_index(axis=1, inplace=True)
    df_output = format_df_output(df)
    colnames = pd.MultiIndex.from_product([col0_names,col1_names])
    index = pd.MultiIndex.from_product([ind0_names, ind1_names])
    df_output.columns = colnames
    df_output.index = index
    df_output.to_excel('{}.xlsx'.format(output_name))
    return df, df_output

In [9]:
df, df_output = create_and_save_df_strat(v_sum_strat_df, 'table_strat_vitesse')